# Inputs

In [1]:
import sys
sys.path.append('../')
import transport_frames.src.graph_builder.criteria as criteria
import transport_frames.src.metrics.indicators as indicators
import transport_frames.src.metrics.grade_territory as grade_territory
import transport_frames.src.graph_builder.graphbuilder as graphbuilder
import momepy
import osmnx as ox
import geopandas as gpd
import shapely
import pandas as pd
import networkx as nx
import numpy as np
import json

/Users/polina/Desktop/github/transport_frames/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def create_json_file(d1, filename):
    json_dict = {}
    for key in d1.keys():
        if isinstance(d1[key],(float,int)):
            json_dict[key] = d1[key]
        else:
            json_dict[key] = d1[key].to_json()
    json_object = json.dumps(json_dict)
    with open(filename, 'w') as f:
        f.write(json_object)

In [3]:
# Ленинградская область EPSG: 32636  /
# Санкт-Петербург EPSG:32636 /
# Москва EPSG:32637 / 
# Волгоградская область EPSG:32638 77665 / нода 27504363
# Тульская область EPSG:32637 / 
# Омская область EPSG:32643 / 
# Краснодарский край EPSG:32637 / 
# Тюменская область EPSG:32642 / 
# Московская область EPSG:32637 / 


In [4]:
region_name = 'Волгоградская_область'
local_crs = 32638
geocode = 77665
region_capital = ox.geocode_to_gdf('N27504363',by_osmid=True)

In [5]:
city = ox.geocode_to_gdf(f'R{geocode}', by_osmid=True).to_crs(epsg=4326)

In [9]:

PLACEHOLDER = gpd.GeoDataFrame(geometry=[]) # set where are no services 

polygons188 = gpd.read_file(f'../data/{region_name}/settlement.geojson')
polygons18 = gpd.read_file(f'../data/{region_name}/district.geojson')

points = gpd.read_file(f'../data/{region_name}/{region_name}_region_points.geojson')
points188 = shapely.centroid(polygons188.geometry).set_crs(polygons188.crs)
points18 = points[points['is_admin_centre_district']==True]


stops = gpd.read_file(f'../data/{region_name}/railway_station.geojson')

fuel = gpd.read_file(f'../data/{region_name}/Копия fuel.geojson')

ferry = PLACEHOLDER

aero = gpd.read_file(f'../data/{region_name}/aerodrome.geojson')

international_aero = aero[aero['aerodrome:type']=='international']
international_aero['geometry'] = shapely.centroid(international_aero['geometry']).set_crs(international_aero.crs)

aero = aero[aero['aerodrome:type']!='international']
aero['geometry'] = shapely.centroid(aero['geometry']).set_crs(aero.crs)


water_objects = PLACEHOLDER

oopt = gpd.read_file(f'../data/{region_name}/Копия nature_reserve.geojson')

train_paths = gpd.read_file(f'../data/{region_name}/railway.geojson')

bus_stops = gpd.read_file(f'../data/{region_name}/bus_stop.geojson')

russia = ox.geocode_to_gdf("Russia") 
regions = gpd.read_file(f'/Users/polina/Desktop/github/transport_frames/Polina/lo_gdfs/regions_of_russia.geojson') #  get regions
regions = regions[regions['ISO3166-2']!='RU-CHU']
regions = regions.to_crs(city.crs)

# test territory
neud = polygons188.iloc[[0]]

# Preprocessing

In [43]:
import os

directory = f"../preprocessed/{region_name}/"
os.makedirs(directory, exist_ok=True)

In [44]:
# write an uds graph
citygraph = graphbuilder.get_graph_from_polygon(city, crs=local_crs,country_polygon=russia)
citygraph_ = graphbuilder.convert_list_attr_to_str(citygraph)
nx.write_graphml(citygraph_, f"../preprocessed/{region_name}/{region_name}_uds.graphml")

/Users/polina/Desktop/github/transport_frames/.venv/lib/python3.10/site-packages/osmnx/_overpass.py:254: UserWarning: This area is 57 times your configured Overpass max query area size. It will automatically be divided up into multiple sub-queries accordingly. This may take a long time.
  multi_poly_proj = utils_geo._consolidate_subdivide_geometry(poly_proj)


In [45]:
# # write inter into graphml
inter = indicators.get_intermodal(geocode,local_crs)
e = [
        (u, v, k)
        for u, v, k, d in inter.edges(data=True, keys=True)
        if d.get("type") in (['bus','walk'])
    ]
inter = inter.edge_subgraph(e)

# save routes and stops
ni,ei = momepy.nx_to_gdf(inter)
bus_routes = ei[ei['type']=='bus']
bus_routes.to_file(f'../preprocessed/{region_name}/{region_name}_bus_routes.geojson')

# convert to file
inter_= graphbuilder.convert_list_attr_to_str(inter)
for node,e,data in inter_.edges(data=True):
    if 'geometry' in data:
        data['geometry']=str(data['geometry'])
nx.write_graphml(inter_, f"../preprocessed/{region_name}/{region_name}_inter.graphml")

2024-06-14 19:53:18.299 | INFO     | dongraphio.dongraphio:get_intermodal_graph_from_osm:61 - Creating intermodal graph from OSM...
2024-06-14 19:53:18.305 | WARNING  | dongraphio.base_models.grapher_logic:get_public_trasport_graph:237 - Files with routes or with stops was not found. The graph will be built based on data from OSM
2024-06-14 19:53:19.789 | WARNING  | dongraphio.base_models.grapher_logic:public_routes_to_edges:162 - It seems there are no subway routes in the city. This transport type will be skipped.
2024-06-14 19:53:49.339 | INFO     | dongraphio.base_models.grapher_logic:get_public_trasport_graph:285 - Public transport graph done!
2024-06-14 19:53:49.757 | DEBUG    | dongraphio.base_models.grapher_logic:get_osmnx_graph:95 - Extracting and preparing walk graph from OSM ...
/Users/polina/Desktop/github/transport_frames/.venv/lib/python3.10/site-packages/osmnx/_overpass.py:254: UserWarning: This area is 55 times your configured Overpass max query area size. It will automa

In [46]:
# write frame into graphml
frame = graphbuilder.get_frame(citygraph,regions.to_crs(city.crs),city)
frame = graphbuilder.assign_city_names_to_nodes(points18.reset_index().to_crs(frame.graph['crs']), 
                                                momepy.nx_to_gdf(frame)[0], frame, name_col='name', max_distance=1200)
frame_ = frame.copy()
for node,e,data in frame_.edges(data=True):
    if 'geometry' in data:
        data['geometry']=str(data['geometry'])
for node,data in frame_.nodes(data=True):
    if 'ref' in data:
        data['ref']=str(data['ref'])
frame_ = graphbuilder.convert_list_attr_to_str(frame_)
nx.write_graphml(frame_, f"../preprocessed/{region_name}/{region_name}_frame.graphml")

/var/folders/ch/1hzm9xv12jg458kt9c3nl6_h0000gn/T/ipykernel_52432/1013225357.py:2: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  frame = graphbuilder.get_frame(citygraph,regions.to_crs(city.crs),city)
/Users/polina/Desktop/github/transport_frames/notebooks/../transport_frames/src/graph_builder/graphbuilder.py:505: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.06]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  n.loc[(n["nodeID"] == path[j]), "weight"] += weight
/Users/polina/Desktop/github/transport_frames/notebooks/../transport_frames/src/graph_builder/graphbuilder.py:506: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.06]' has dtype incompatible with int64, please explicitly cast to a c

# Processing

In [10]:
citygraph = nx.read_graphml(f'../preprocessed/{region_name}/{region_name}_uds.graphml')
citygraph = graphbuilder.convert_list_attr_from_str(citygraph)
citygraph = indicators.prepare_graph(citygraph)

In [11]:
inter = nx.read_graphml(f"../preprocessed/{region_name}/{region_name}_inter.graphml")
inter = graphbuilder.convert_list_attr_from_str(inter)
inter = indicators.prepare_graph(inter)

In [13]:
bus_routes = gpd.read_file(f'../preprocessed/{region_name}/{region_name}_bus_routes.geojson')

# # or if we don't have them:
# ni,ei = momepy.nx_to_gdf(inter)
# bus_routes = ei[ei['type']=='bus']

In [14]:
frame = nx.read_graphml(f"../preprocessed/{region_name}/{region_name}_frame.graphml")
frame = graphbuilder.convert_list_attr_from_str(frame)
frame = indicators.prepare_graph(frame)

In [ ]:
d0 = indicators.indicator_area(citygraph, city, points, inter,region_capital, 
                               fuel, stops, international_aero, aero, ferry,train_paths,bus_stops, crs=local_crs)
for k,v in d0.items():
    if isinstance(v,gpd.GeoDataFrame):
        cols_to_drop = [col for col in v.columns if col not in ['index', 'name', 'layer', 'density', 'geometry', 'reg1_length', 'reg2_length', 'reg3_length', 'to_service', 'index', 'number_of_routes', 'service_count', 'total_length_km']]
        v.drop(cols_to_drop, axis=1, inplace=True)
create_json_file(d0,f"../preprocessed/{region_name}/{region_name}_indicators_region.json")

In [ ]:
d1 = indicators.indicator_area(citygraph, polygons18, points, inter,region_capital, fuel, stops, international_aero, aero, ferry,train_paths,bus_stops, crs=local_crs)

for k,v in d1.items():
    if isinstance(v,gpd.GeoDataFrame):
        cols_to_drop = [col for col in v.columns if col not in ['index', 'name', 'layer', 'density', 'geometry', 'reg1_length', 'reg2_length', 'reg3_length', 'to_service', 'index', 'number_of_routes', 'service_count', 'total_length_km']]
        v.drop(cols_to_drop, axis=1, inplace=True)
create_json_file(d1,f"../preprocessed/{region_name}/{region_name}_indicators_mr.json")

In [ ]:
d2 = indicators.indicator_area(citygraph, polygons188, points, inter,region_capital, 
                               fuel, stops, international_aero, aero, ferry,train_paths,bus_stops, crs=local_crs)

for k,v in d2.items():
    if isinstance(v,gpd.GeoDataFrame):
        cols_to_drop = [col for col in v.columns if col not in ['index', 'name', 'layer', 'density', 'geometry', 'reg1_length', 'reg2_length', 'reg3_length', 'to_service', 'index', 'number_of_routes', 'service_count', 'total_length_km']]
        v.drop(cols_to_drop, axis=1, inplace=True)
create_json_file(d2,f"../preprocessed/{region_name}/{region_name}_indicators_gpsp.json")

In [ ]:
d3 = indicators.indicator_territory(citygraph, neud,
                                    regions_gdf=polygons18,
                                    districts_gdf=polygons188,
                                    region_capital=region_capital,
                                    region_centers=points18,
                                    district_centers=polygons188,
                                    settlement_centers=points,
                                    fuel=fuel,
                                    train_stops=stops,
                                    international_aero=international_aero,
                                    aero=aero,
                                    ports=ferry,
                                    water_objects=water_objects, 
                                    oopt=oopt, 
                                    train_paths=train_paths, 
                                    inter = None,
                                    bus_stops=bus_stops, 
                                    bus_routes=bus_routes,crs=local_crs)